In [2]:
! pip3 install torch torchvision

In [1]:
from torchvision import datasets, transforms 
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import resnet101, ResNet101_Weights
from tempfile import TemporaryDirectory
from matplotlib import pyplot as plt
import torch.nn as nn
import torch.utils.data
import numpy as np
import os, time
from torchvision.datasets import OxfordIIITPet
from torchvision.models import efficientnet_b4, EfficientNet_B4_Weights
from torchvision.models import efficientnet_b7, EfficientNet_B7_Weights
from torchvision.models import efficientnet_v2_m, EfficientNet_V2_M_Weights

In [91]:
def show(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), interpolation='nearest')
    plt.show()

In [92]:
def test_model(model, test_data, weights_path, device_type='cpu'):
    if torch.cuda.is_available():
        device_type = 'cuda:0'
    elif torch.backends.mps.is_available():
        device_type = 'mps'

    device = torch.device(device_type)
    model.load_state_dict(torch.load(weights_path))
    model = model.to(device)

    start = time.time()
    with torch.no_grad():
        model.eval()
        correct = 0
        total = 0
        for images, labels in test_data:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)

            predicted = torch.argmax(outputs, 1)

            total += labels.size(0)
            correct += torch.sum(predicted == labels.data)
        print('Time taken:', time.time() - start)
        acc = 100 * correct / total
        print(f'Accuracy of the network on the test images: {100 * correct / total}%')
    
    return acc

In [1]:
def train_model(
        model, dataloaders, dataset_sizes,
        suffix, parameters, scheduler=0,
        num_epochs=25, device_type='cpu',
        lr_params=None, optimizer_name='sgd',
        filename='training.log'):
    if torch.cuda.is_available():
        device_type = 'cuda'
    elif torch.backends.mps.is_available():
        device_type = 'mps'

    device = torch.device(device_type)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    if optimizer_name == 'adam':
        if lr_params is None:
            optimizer = torch.optim.Adam(parameters, lr=0.001)
        else:
            optimizer = torch.optim.Adam(params=lr_params)
    elif optimizer_name == 'sgd':
        if lr_params is None:
            optimizer = torch.optim.SGD(parameters, lr=0.001, momentum=0.9)
        else:
            optimizer = torch.optim.SGD(params=lr_params, momentum=0.9)

    since = time.time()

    losses = {'train': [], 'val': []}
    accuracies = {'train': [], 'val': []}

    # Create a directory to save training checkpoints
    with TemporaryDirectory(), open(filename, 'w') as f:
        best_model_params_path = os.path.join("./", suffix + '.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        start = time.time()

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)
            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                model.train() if phase == 'train' else model.eval()

                running_loss = 0.0
                running_corrects = 0

                print('phase:', phase)
                
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs) # forward pass

                        # outputs is probability of each class
                        # labels needs to be probability of each class

                        # we need the one-hot for the labels to get the loss
                        true_outputs = torch.nn.functional.one_hot(labels, num_classes=37).float()

                        # create preds by thresholding outputs
                        loss = criterion(outputs, true_outputs)

                        preds = torch.argmax(outputs, 1)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects / dataset_sizes[phase]

                losses[phase].append(epoch_loss)
                accuracies[phase].append(epoch_acc.cpu())
                
                outputstr = f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}'
                print(outputstr)
                f.write(outputstr + '\n')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)
            print('epoch took:', time.time() - start)
            start = time.time()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return best_model_params_path, best_acc, losses, accuracies

In [94]:
from torch.utils.data import Subset

def get_transforms(conf):
    base_transforms = [
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]

    training_transforms = []

    crop = conf.get('crop', False)
    random_flip = conf.get('random_flip', False)
    rotate_angle = conf.get('rotate_angle', 0)
    random_resized_crop = conf.get('random_resized_crop', False)

    if random_resized_crop:
        training_transforms.append(transforms.RandomResizedCrop(224))
    elif crop:
        base_transforms.append(transforms.Resize(256))
        base_transforms.append(transforms.CenterCrop(224))

    if random_flip:
        training_transforms.append(transforms.RandomHorizontalFlip())
    if rotate_angle != 0:
        training_transforms.append(transforms.RandomRotation(rotate_angle))
    
    return base_transforms, training_transforms

def train_test_val_split(base_transforms, training_transforms, split_indices):
    train_dataset=OxfordIIITPet(
        root="./", download=True, target_types='category',
        transform=transforms.Compose(base_transforms + training_transforms + [transforms.Resize((224, 224))])
        )

    val_dataset=OxfordIIITPet(
        root="./", download=True, target_types='category',
        transform=transforms.Compose(base_transforms + [transforms.Resize((224, 224))])
        )
    
    test_dataset=OxfordIIITPet(
        root="./", download=True, target_types='category', split='test',
        transform=transforms.Compose(base_transforms + [transforms.Resize((224, 224))])
        )

    train_dataset = Subset(train_dataset, split_indices['train'])
    val_dataset = Subset(val_dataset, split_indices['val'])
    
    return train_dataset, val_dataset, test_dataset

def get_split_indices_train_val(dataset=OxfordIIITPet(root="./", download=True, target_types='category'), train_split=0.8, val_split=0.2, seed=None):
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    
    if seed is not None:
        np.random.seed(42)
    np.random.shuffle(indices)
    
    train_size = int(train_split * dataset_size)
    
    # Split indices
    train_indices = indices[:train_size]
    val_indices = indices[train_size:]
    
    # Save the indices for reuse
    split_indices = {
        'train': train_indices,
        'val': val_indices,
    }
    
    return split_indices

def generate_datasets(
        conf, split_indices
    ):

    base_transforms, training_transforms = get_transforms(conf)
    train_dataset, val_dataset, test_dataset = train_test_val_split(base_transforms, training_transforms, split_indices)

    dataloaders = {
        'train': torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True),
        'test': torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False),
        'val': torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False),
    }

    dataset_sizes = {
        'train': len(train_dataset),
        'test': len(test_dataset),
        'val': len(val_dataset),
    }

    return dataloaders, dataset_sizes

In [95]:
def multiclass_resnet(name = "resnet18"):
    if name.endswith("18"):
        resnet = resnet18(weights=ResNet18_Weights.DEFAULT)
    elif name.endswith("50"):
        resnet = resnet50(weights=ResNet50_Weights.DEFAULT)
    elif name.endswith("101"):
        resnet = resnet101(weights=ResNet101_Weights.DEFAULT)
    
    for param in resnet.parameters():
        param.requires_grad = False
    
    print(resnet.fc)

    #resnet fc input size
    input_size = resnet.fc.in_features

    resnet.fc = nn.Sequential(
        # nn.dropout(0.3),
        nn.Linear(input_size, 37)
    )

    return resnet

def multiclass_efficientnet(name = "efficientnet_b4"):
    if name == "efficientnet_b4":
        efficientnet = efficientnet_b4(weights=EfficientNet_B4_Weights.DEFAULT)
    elif name == "efficientnet_v2_m":
        efficientnet = efficientnet_v2_m(weights=EfficientNet_V2_M_Weights.DEFAULT)
    elif name == "efficientnet_b7":
        efficientnet = efficientnet_b7(weights=EfficientNet_B7_Weights.DEFAULT)
    
    for param in efficientnet.parameters():
        param.requires_grad = False
    
    print(efficientnet.classifier)

    #efficientnet fc input size
    input_size = efficientnet.classifier[1].in_features

    efficientnet.classifier = nn.Sequential(
        nn.Dropout(0.3),
        nn.Linear(input_size, 37)
    )

    return efficientnet

In [102]:
def create_suffix(
        conf,
        unfrozen_layers=None
    ):
    device_type = 'cpu'
    if torch.cuda.is_available():
        device_type = 'cuda'
    elif torch.backends.mps.is_available():
        device_type = 'mps'
    
    suffix = ""
    for key, value in conf.items():
        suffix += key + ':' + str(value) + ';'
    suffix += 'device:' + device_type
    if unfrozen_layers is not None:
        suffix += 'unfrozen:' + str(unfrozen_layers)

    return suffix

In [97]:
model_name = 'efficientnet_v2_m'
random_flip = False
rotate_angle = 0
crop = False
random_resized_crop = 1

base_conf = {
    'model_name': model_name,
    'crop': crop,
    'random_flip': random_flip,
    'rotate_angle': rotate_angle,
    'scale': random_resized_crop
}


In [98]:
def get_model(model_name):
    if model_name.startswith('resnet'):
        return multiclass_resnet(model_name)
    return multiclass_efficientnet(model_name)

## Model tests

In [ ]:
split_indices = get_split_indices_train_val(seed=42)

In [ ]:
# model_names = ['resnet18', 'resnet50', 'efficientnet_b4', 'efficientnet_v2_m']
model_names = ['resnet101', 'efficientnet_b4']

losses_and_accs_models = {}
total_best_acc = 0
best_acc_weights_path = None

for model_name in model_names:
    conf = {
        'model_name': model_name,
        'crop': False,
        'random_flip': False,
        'rotate_angle': 0,
        'random_resized_crop': False
    }

    dataloaders, dataset_sizes = generate_datasets(conf, split_indices)
    model = get_model(model_name)
    parameters = model.fc.parameters() if conf['model_name'].startswith('resnet') else model.classifier.parameters()
    suffix = create_suffix(conf)
    print('-' * 20)
    print('Training model:', model_name)
    weights_path, best_acc, losses, accuracies =\
        train_model(model, dataloaders, dataset_sizes,
                    suffix, parameters=parameters,
                    num_epochs=20, filename=f'{model_name}_training.log')
    if best_acc > total_best_acc:
        total_best_acc = best_acc
        best_acc_weights_path = weights_path
    losses_and_accs_models[model_name] = (losses, accuracies)
    print('-' * 20)
print('Accuracy for best config combination:', total_best_acc)
print('Weights path for best combination', best_acc_weights_path)
    

Plot model losses and accuracies

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
for model_name, (losses, accuracies) in losses_and_accs_models.items():
    plt.plot(losses['train'], label=f'{model_name} train')
    plt.plot(losses['val'], label=f'{model_name} val')
    plt.title('Loss plot for testing model: ' + model_name)
    plt.legend()
    plt.show()
    plt.savefig(f'loss_plot_{model_name}.png')

    plt.plot(accuracies['train'], label=f'{model_name} train')
    plt.plot(accuracies['val'], label=f'{model_name} val')
    plt.title('Accuracy plot for testing model: ' + model_name)
    plt.legend()
    plt.show()
    plt.savefig(f'acc_plot_{model_name}.png')

## Data augmentation experiments

In [103]:
def generate_confs():
    model_name = 'resnet50'
    crops = ['off', 'center-crop', 'random-resized-crop']
    random_flips = [True, False]
    rotate_angles = [0, 30]

    confs = []
    for crop in crops:
        for random_flip in random_flips:
            for rotate_angle in rotate_angles:
                conf = {
                    'model_name': model_name,
                    'crop': crop == 'center-crop',
                    'random_flip': random_flip,
                    'rotate_angle': rotate_angle,
                    'random_resized_crop': crop == 'random-resized-crop',
                }
                confs.append(conf)

    return confs

confs = generate_confs()

### Run config tests

In [104]:
losses_and_accs_configs = []
total_best_acc = 0
best_acc_weights_path = None
for conf in confs:
    dataloaders, dataset_sizes = generate_datasets(conf, split_indices)
    model = get_model(conf['model_name'])
    suffix = create_suffix(conf)
    parameters = model.fc.parameters() if conf['model_name'].startswith('resnet') else model.classifier.parameters()

    print('-' * 20)
    print('Training model with params:', conf)

    weights_path, best_acc, losses, accuracies =\
        train_model(model, dataloaders, dataset_sizes, suffix, parameters=parameters, num_epochs=10, filename=f'training_{suffix}.txt')
    if best_acc > total_best_acc:
        total_best_acc = best_acc
        best_acc_weights_path = weights_path
    
    losses_and_accs_configs.append((suffix, losses, accuracies))
    print('-' * 20)

print('Accuracy for best config combination:', total_best_acc)
print('Weights path for best combination', best_acc_weights_path)

Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)
--------------------
Training model with params: {'model_name': 'efficientnet_v2_m', 'crop': False, 'random_flip': True, 'rotate_angle': 0, 'random_resized_crop': False}
Epoch 0/4
----------
phase: train


KeyboardInterrupt: 

Plot config tests 

In [ ]:
for (suffix, losses, accuracies) in losses_and_accs_configs:
    plt.plot(losses['train'], label='train')
    plt.plot(losses['val'], label='val')
    plt.title('Loss plot for configs: ' + ', '.join(suffix.split(';')))
    plt.legend()
    plt.show()
    # save plot
    plt.savefig(suffix + '.png')

    plt.plot(accuracies['train'], label='train')
    plt.plot(accuracies['val'], label='val')
    plt.title('Val accuracy plot for configs: ' + ', '.join(suffix.split(';')))
    plt.legend()
    plt.show()
    plt.savefig(suffix + '.png')

## Train more layers experiments

In [54]:
# unfrozen_layers are the number of layers that are unfrozen
def get_model_frozen_until(unfrozen_layers, model_name):
    model = get_model(model_name)
    children = list(model.children())
    params_lr = []

    for child in children[-unfrozen_layers:]:
        for param in child.parameters():
            param.requires_grad = True 

    lr = 0.1

    for child in children[-unfrozen_layers:]:
        params_lr.append({'params': child.parameters(), 'lr': lr})
        lr /= 10
        
    return model, params_lr

In [3]:
losses_and_accs_unfrozen = []
total_best_acc = 0
best_acc_weights_path = None
model_name = 'resnet101'

dataloaders, dataset_sizes = generate_datasets(base_conf, split_indices)

for unfrozen_layers in range(1, 6):
    model, _ = get_model_frozen_until(unfrozen_layers, model_name)
    suffix = create_suffix(conf, unfrozen_layers)
    # parameters = model.fc.parameters() if model_name.startswith('resnet') else model.classifier.parameters()
    parameters = model.parameters()
    print('training model with', unfrozen_layers, 'unfrozen layers')
    weights_path, best_acc, losses, accuracies =\
        train_model(model, dataloaders, dataset_sizes, suffix, parameters=parameters, num_epochs=10, filename=f'unfreeze_{unfrozen_layers}.txt')
    if best_acc > total_best_acc:
        total_best_acc = best_acc
        best_acc_weights_path = weights_path
    losses_and_accs_unfrozen.append((unfrozen_layers, losses, accuracies))

print('Accuracy for best config combination:', total_best_acc)
print('Weights path for best combination', best_acc_weights_path)

NameError: name 'generate_datasets' is not defined

Plot train more layers 

In [ ]:
for (unfrozen_layers, losses, accuracies) in losses_and_accs_unfrozen:
    plt.plot(losses['train'], label='train')
    plt.plot(losses['val'], label='val')
    plt.title('Loss plot for unfrozen layers: ' + str(unfrozen_layers))
    plt.legend()
    plt.show()
    plt.savefig(f'unfrozen_{unfrozen_layers}.png')

    plt.plot(accuracies['train'], label='train')
    plt.plot(accuracies['val'], label='val')
    plt.title('Val accuracy plot for unfrozen layers: ' + str(unfrozen_layers))
    plt.legend()
    plt.show()
    plt.savefig(f'unfrozen_{unfrozen_layers}.png')

## Learning rate tests

In [ ]:
losses_and_accs_unfrozen = []
total_best_acc = 0
best_acc_weights_path = None
model_name = 'resnet101'

for unfrozen_layers in range(1, 6):
    model, params_lr = get_model_frozen_until(unfrozen_layers, model_name)
    suffix = create_suffix(conf, unfrozen_layers)
    parameters = model.parameters()
    print('training model with', unfrozen_layers, 'unfrozen layers')
    weights_path, best_acc, losses, accuracies =\
        train_model(model, dataloaders,
                    dataset_sizes, suffix,
                    parameters=parameters, num_epochs=10,
                    lr_params=params_lr, filename=f'lrs_{unfrozen_layers}.txt')
    if best_acc > total_best_acc:
        total_best_acc = best_acc
        best_acc_weights_path = weights_path
    losses_and_accs_unfrozen.append((unfrozen_layers, losses, accuracies))

print('Accuracy for best config combination:', total_best_acc)
print('Weights path for best combination', best_acc_weights_path)

## Final test

In [ ]:
best_conf = {
    'model_name': 'resnet101',
    'crop': False,
    'random_flip': False,
    'rotate_angle': 0,
    'random_resized_crop': True
}

dataloaders, dataset_sizes = generate_datasets(base_conf, split_indices)
model, _ = get_model_frozen_until(5, best_conf['model_name'])
parameters = model.parameters()
suffix = create_suffix(best_conf, 5)
weights_path, best_acc, losses, accuracies =\
        train_model(model, dataloaders,
                    dataset_sizes, suffix,
                    parameters=parameters, num_epochs=25,
                    filename='best_model_results.txt')

print('Best acc for final model: ', best_acc)

In [ ]:
acc = test_model(model, dataloaders['test'], weights_path)